In [1]:
import json
import os
import openai
import requests


In [2]:
# Load config values
with open('config.json', 'r', encoding='utf-16') as file:
    config_details = json.load(file)
    
# Setting up the deployment name
deployment_name = config_details['COMPLETIONS_MODEL']

# The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
openai_api_base = config_details['OPENAI_API_BASE']

# The API key for your Azure OpenAI resource.
openai_api_key  = config_details['OPENAI_API_KEY']
#print(openai_api_key)

# Currently OPENAI API have the following versions available: 2022-12-01. All versions follow the YYYY-MM-DD date structure.
openai_api_version = config_details['OPENAI_API_VERSION']

In [3]:
# Request URL
def request_response_from_model(messages):
    api_url = f"{openai_api_base}/openai/deployments/{deployment_name}/chat/completions?api-version={openai_api_version}"
    # Example prompt for request payload    
    
    # Json payload
    # To know more about the parameters, checkout this documentation: https://learn.microsoft.com/en-us/azure/cognitive-services/openai/reference
    json_data = {
      "messages": messages,
      "temperature":0,
      "max_tokens": 500
    }
    
    # Including the api-key in HTTP headers
    headers =  {"api-key": openai_api_key}
    
    try:
        # Request for creating a completion for the provided prompt and parameters
        response = requests.post(api_url, json=json_data, headers=headers)
        completion = response.json()

        # return response
        return (completion['choices'][0]['message']['content'])
    except:
        return ("Error Message:", completion['error']['message'])

In [4]:
unread_chat_messages = {
  "chat": [
    {"user": "Alice", "message": "Hi Bob, hope you're doing well. We're just a few days away from the Project Unicorn release on 5th Jan 2024. Let's go over the last-minute work items we need to handle."},
    {"user": "Bob", "message": "Hi Alice, I'm good, thanks. Yes, the deadline is approaching fast. I've been reviewing the tasks, and it looks like we still have a few outstanding items."},
    {"user": "Alice", "message": "Great. Could you please share the status of the critical tasks? Also, remember to provide work item links for reference."},
    {"user": "Bob", "message": "Sure. Task 123 - \"UI Enhancement\" is almost done. I just need to address some minor tweaks. [Link](https://ADO/WI/UI_Enhancement) Due Date: 3rd Jan 2024, Visibility: All Users"},
    {"user": "Alice", "message": "Good progress on that one. What about Task 456 - \"Database Optimization\"?"},
    {"user": "Bob", "message": "That's in progress, but I've encountered a few unexpected challenges. I'll update you once I have a clearer picture. [Link](https://ADO/WI/Database_Optimization) Due Date: 4th Jan 2024, Visibility: Backend Team"},
    {"user": "Alice", "message": "Alright, keep me posted on that. We can allocate additional resources if needed. Also, Task 789 - \"Testing Automation\"?"},
    {"user": "Bob", "message": "Testing Automation is on track. The test cases are ready, and I'll start the automation script development today. [Link](https://ADO/WI/Testing_Automation) Due Date: 4th Jan 2024, Visibility: QA Team"},
    {"user": "Alice", "message": "Excellent, Bob. That's a critical aspect for a smooth release. Now, Task 101 - \"Documentation Update\"?"},
    {"user": "Bob", "message": "I've updated the user documentation, but I need your input on the release notes. Could you please review and provide your comments? [Link](https://ADO/WI/Documentation_Update) Due Date: 3rd Jan 2024, Visibility: Documentation Team"},
    {"user": "Alice", "message": "I'll take a look at it today and get back to you. Finally, Task 234 - \"Performance Testing\"?"},
    {"user": "Bob", "message": "Performance testing is underway. I've identified a potential bottleneck, and I'm working with the team to address it. [Link](https://ADO/WI/Performance_Testing) Due Date: 4th Jan 2024, Visibility: DevOps Team"},
    {"user": "Alice", "message": "Good catch, Bob. Let's prioritize resolving that issue. Keep me informed of any roadblocks. We can't afford performance issues in the release."},
    {"user": "Bob", "message": "Absolutely, Alice. I'll keep you updated, and we'll ensure everything is in top shape before the 5th Jan deadline."},
    {"user": "Alice", "message": "Thanks, Bob. Let's aim for a smooth release. If you need any additional support, don't hesitate to reach out. We're in the final stretch!"},
    {"user": "Bob", "message": "Will do, Alice. Let's make Project Unicorn a success!"},
    {"user": "Alice", "message": "Bob, I've just assigned two new tasks to you. Task 567 - \"Security Review\" [Link](https://ADO/WI/Security_Review) Due Date: 4th Jan 2024, Visibility: Security Team and Task 890 - \"Bug Fixes\" [Link](https://ADO/WI/Bug_Fixes) Due Date: 3rd Jan 2024, Visibility: Development Team. Please prioritize these alongside the existing tasks."},
    {"user": "Bob", "message": "Got it, Alice. I'll make sure to address these new tasks promptly. Thanks for the heads up!"}
  ]
}

In [10]:
delimiter = "####"
system_message = f"""
You are a chat summarization assistant named Chatty. \
You will be provided with chat messages. \
The chat messages will be delimited with \
{delimiter} characters.
Output a json object of the following format \
{{
 "summarization": <summarization of all the messages given as input>,
 "action_items": <[ action_item1, action_item2, ..., action_itemN]>
}}

Where summarization of the messages must be given \
from the input messages given as input \
and with the tone of summarizing of missed messages when user was offline.
For listing the action_items refer to the message content \
inside input and list out \
items that have tasks associated with it \
and add the link of the task items as well. \

Sample action_items list: \
"action_items": [ 
    {{
        item: action_item1,
        due_date: date1,
        visibility_score: All users
    }},
    {{
        item: action_item2,
        due_date: date2,
        visibility_score: Backend Team
    }}
]

Only output the json object, with nothing else.
"""
user_message_1 = f"""
 Can you help summize my unread messages given below \
 {unread_chat_messages}
 """
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message_1}{delimiter}"},  
] 

chatty_summarization_response = request_response_from_model(messages)
print(chatty_summarization_response)

{
 "summarization": "Alice and Bob discussed the status of the tasks related to the Project Unicorn release scheduled for 5th Jan 2024. Bob provided updates on the progress of the tasks and shared links for reference. Alice assigned two new tasks to Bob and asked him to prioritize them along with the existing tasks.",
 "action_items": [
    {
        "item": "UI Enhancement",
        "due_date": "3rd Jan 2024",
        "visibility_score": "All Users",
        "link": "https://ADO/WI/UI_Enhancement"
    },
    {
        "item": "Database Optimization",
        "due_date": "4th Jan 2024",
        "visibility_score": "Backend Team",
        "link": "https://ADO/WI/Database_Optimization"
    },
    {
        "item": "Testing Automation",
        "due_date": "4th Jan 2024",
        "visibility_score": "QA Team",
        "link": "https://ADO/WI/Testing_Automation"
    },
    {
        "item": "Documentation Update",
        "due_date": "3rd Jan 2024",
        "visibility_score": "Documentati